In [2]:
import pickle as pkl
import pandas as pd
import numpy as np
import random

random.seed(1)

In [3]:
labels = pkl.load(open("./Data/Learn/labels.pkl", "rb"))
labels = pd.Series(labels)
print(labels.head())
print(labels.count())
print(labels.value_counts())

0    C
1    C
2    C
3    C
4    C
dtype: object
45930
C    39912
M     6018
dtype: int64


In [4]:
labels[39912]

'M'

In [5]:
sentences = pkl.load(open("./Data/Learn/sentences.pkl", "rb"))
print(sentences[:5])
print(len(sentences))

["J'aurai l'occasion de dire aux Français comment notre enseignement devra évoluer pour permettre à chaque jeune de trouver sa place, d'entrer dans le monde du travail, de savoir s'adapter et, à partir de là, d'acquérir, tout au long de la vie, de nouvelles compétences et de nouveaux savoirs.", 'Il est nécessaire.', "Dans votre coeur et dans votre vie, la confiance et l'enthousiasme l'emportent sur le doute.", "Pour conduire ce débat dans un esprit de véritable dialogue, je compte, si nos partenaires en sont d'accord, inviter au prochain sommet du G7, à Lyon, pour une séance de travail, le secrétaire général des Nations Unies, le président de la Banque mondiale et le directeur général du Fonds monétaire international.", "La France et l'Europe construiront ainsi un avenir de coopération avec un Proche-Orient pacifié, stable, prospère, libéré des menaces de la guerre, de la prolifération, du terrorisme."]
45930


In [6]:
sequences = pkl.load(open("./Data/Learn/sequences.pkl", "rb"))
print(sequences[:5])
print(len(sequences))

[[54, 2187, 5, 320, 1, 102, 47, 74, 357, 33, 640, 613, 1377, 17, 434, 6, 144, 834, 1, 452, 69, 127, 0, 10, 1604, 18, 7, 71, 19, 152, 0, 1, 359, 34, 683, 4, 0, 6, 800, 1, 128, 0, 10, 2741, 0, 52, 25, 474, 1, 3, 99, 0, 1, 205, 880, 4, 1, 313, 2428, 2], [21, 11, 279, 2], [18, 53, 165, 4, 18, 53, 99, 0, 3, 169, 4, 5, 1466, 5, 6065, 35, 7, 364, 2], [17, 848, 27, 554, 18, 15, 175, 1, 425, 163, 0, 20, 255, 0, 77, 38, 273, 12, 44, 10, 351, 0, 4791, 25, 604, 342, 19, 1629, 0, 6, 1592, 0, 17, 16, 5191, 1, 152, 0, 7, 1065, 182, 9, 248, 567, 0, 7, 82, 1, 3, 1291, 406, 4, 7, 2036, 182, 19, 789, 1040, 461, 2], [3, 37, 4, 5, 68, 17747, 146, 15, 111, 1, 195, 39, 15, 699, 24, 912, 15240, 0, 2232, 0, 2858, 0, 4800, 9, 1810, 1, 3, 382, 0, 1, 3, 2384, 0, 19, 961, 2]]
45930


In [7]:
vocab = pkl.load(open("./Data/dict.pkl", "rb"))
print(len(vocab))
print(vocab)

30432
{',': 0, 'de': 1, '.': 2, 'la': 3, 'et': 4, "l'": 5, 'à': 6, 'le': 7, 'les': 8, 'des': 9, "d'": 10, 'est': 11, 'en': 12, 'que': 13, 'qui': 14, 'un': 15, 'une': 16, 'pour': 17, 'dans': 18, 'du': 19, 'je': 20, 'il': 21, 'nous': 22, 'vous': 23, '-': 24, 'au': 25, 'a': 26, 'ce': 27, "c'": 28, 'plus': 29, "qu'": 30, 'pas': 31, 'par': 32, 'notre': 33, "s'": 34, 'sur': 35, 'ne': 36, 'france': 37, 'nos': 38, 'avec': 39, 'cette': 40, 'se': 41, 'mais': 42, 'pays': 43, 'sont': 44, 'aussi': 45, 'elle': 46, 'aux': 47, "n'": 48, 'ont': 49, 'être': 50, 'leur': 51, 'tout': 52, 'votre': 53, "j'": 54, 'tous': 55, 'son': 56, 'y': 57, '"': 58, 'on': 59, 'bien': 60, 'ces': 61, 'même': 62, 'ses': 63, 'ou': 64, 'comme': 65, 'entre': 66, ':': 67, 'europe': 68, 'sa': 69, "aujourd'hui": 70, 'monde': 71, 'doit': 72, 'faire': 73, 'français': 74, 'ai': 75, 'ils': 76, 'si': 77, 'faut': 78, 'sans': 79, 'été': 80, 'fait': 81, 'président': 82, 'où': 83, 'leurs': 84, 'cela': 85, 'avez': 86, 'très': 87, 'monsieur'

In [8]:
def sentence_generator(labels, sentences):
    chirac = random.randint(0, 39912)
    mitterand = random.randint(39912, len(labels))
    print("Chirac: ", sentences[chirac])
    print("Mitterand: ", sentences[mitterand])
sentence_generator(labels, sentences)

Chirac:  C'est un très grand plaisir pour moi de participer avec vous à la clôture de ce Congrès mondial de la comptabilité.
Mitterand:  Bien entendu, cet excès du langage, à la fois, recouvre une certaine exaltation inutile de l'esprit, mais d'autre part, il arrive que cela soit vrai.


In [9]:
import torch
import torch.autograd as autograd
import torch.utils.data as data

In [10]:
torch.cuda.is_available()

True

In [11]:
def train_test_split(labels, sentences, sequences, train_ratio=0.75):
    nb_chirac, nb_mitt = pd.Series(labels).value_counts().values
    print(nb_chirac, nb_mitt)
    nb_chirac_train = nb_chirac * train_ratio
    nb_mitt_train = nb_mitt * train_ratio
    
    

    
train_test_split(labels, sentences, sequences)

39912 6018


In [20]:
def build_weights(labels):
    weights = np.ones(len(labels))
    nb_chirac, nb_mitt = pd.Series(labels).value_counts().values
    print(nb_chirac / (len(labels)))
    weights[:nb_chirac] /= nb_chirac
    weights[nb_chirac:] /= nb_mitt
    return weights

print(build_weights(labels)[39911:39913])

0.8689745264532985
[2.50551213e-05 1.66168162e-04]


In [19]:
random.seed(1)

weights = build_weights(labels)

# create training and test datasets
train_ratio = 0.70
train_indices = random.choices(weights=weights, population=range(len(labels)), k=int(len(labels)*train_ratio))
print(train_indices)


tensor_seq = [autograd.Variable(torch.LongTensor(seq).unsqueeze(0)) for seq in sequences]
print(tensor_seq[:5])
dataset = data.TensorDataset(*tensor_seq)
sampler = data.WeightedRandomSampler(weights=weights, num_samples=1)

[10725, 44093, 43086, 20360, 39547, 35880, 41736, 43387, 7492, 2262, 43953, 34545, 43068, 168, 35552, 42578, 18260, 45271, 44743, 2441, 2031, 40410, 45197, 30429, 17289, 33695, 2318, 17696, 34953, 39577, 18605, 18428, 17463, 36687, 23131, 1715, 43975, 40591, 41624, 14839, 45840, 44244, 9649, 26557, 42577, 42453, 45164, 33694, 43884, 41961, 24216, 40966, 44515, 44078, 39975, 40983, 2755, 19376, 43491, 33072, 13810, 40499, 42355, 42012, 29910, 35039, 40013, 43263, 40164, 31391, 39089, 2360, 3471, 42359, 45727, 41033, 31418, 13597, 39938, 45714, 43168, 40388, 44248, 18533, 40077, 45357, 40848, 36649, 21494, 40489, 45413, 455, 43326, 43769, 44560, 42806, 43632, 40136, 40650, 34012, 4479, 44365, 40754, 15951, 39968, 38708, 28480, 27625, 40375, 41398, 41265, 36571, 2233, 18327, 14145, 40928, 44257, 43504, 43487, 43720, 20378, 44025, 41995, 6644, 1332, 1162, 42988, 19920, 8739, 41414, 27493, 5548, 12741, 40241, 13422, 21785, 42458, 36296, 25703, 37818, 1886, 30856, 33599, 15010, 8681, 44724, 

[tensor([[  54, 2187,    5,  320,    1,  102,   47,   74,  357,   33,  640,  613,
         1377,   17,  434,    6,  144,  834,    1,  452,   69,  127,    0,   10,
         1604,   18,    7,   71,   19,  152,    0,    1,  359,   34,  683,    4,
            0,    6,  800,    1,  128,    0,   10, 2741,    0,   52,   25,  474,
            1,    3,   99,    0,    1,  205,  880,    4,    1,  313, 2428,    2]]), tensor([[ 21,  11, 279,   2]]), tensor([[  18,   53,  165,    4,   18,   53,   99,    0,    3,  169,    4,    5,
         1466,    5, 6065,   35,    7,  364,    2]]), tensor([[  17,  848,   27,  554,   18,   15,  175,    1,  425,  163,    0,   20,
          255,    0,   77,   38,  273,   12,   44,   10,  351,    0, 4791,   25,
          604,  342,   19, 1629,    0,    6, 1592,    0,   17,   16, 5191,    1,
          152,    0,    7, 1065,  182,    9,  248,  567,    0,    7,   82,    1,
            3, 1291,  406,    4,    7, 2036,  182,   19,  789, 1040,  461,    2]]), tensor([[    3, 

In [17]:
random.seed(1)
random.choices(tensor_seq, weights=weights)

[tensor([[    5,   220,     1,  1729,    36, 15886,    31,    66, 13152,     4,
             96,     0,    66,     3, 22861,     4,     7, 22862,     2]])]